In [1]:
import spacy
import pandas as pd

test_file_path = 
# test_df = pd.read_csv('data/data-orig/4_test_participant.csv')
# test_df.head()

In [2]:
# load in training data
#val_fid = 'data/task4_ekp_test.jsonl'
val_fid = 'data/forprodigy/val-orig-ekp.jsonl'


In [3]:
val_df = pd.read_json(val_fid, lines=True)

In [4]:
val_df['tweetid'] = val_df['metadata'].map(lambda x: x.get('tweetid'))

In [5]:
val_df.head()

,text,metadata,label,tweetid
0,can somebody tell me what morphine is for ?,{'tweetid': 1201409307167862784},MENTION,1201409307167862784
1,<number> mg . <number> of my x <number> i feel...,{'tweetid': 1200007750383738885},MENTION,1200007750383738885
2,"oh hello crippling anxiety , let ’ s drive hom...",{'tweetid': 1199244035006902272},CONSUMPTION,1199244035006902272
3,pop a adderall we gone fuck all night,{'tweetid': 1199782125609902084},MENTION,1199782125609902084
4,<hashtag> fake exercise facts </hashtag> a xan...,{'tweetid': 1199783941764517889},MENTION,1199783941764517889


In [6]:
mid = 'models/finalsm4hmodel/'
print(f"loading {mid}")
nlp = spacy.load(mid)

loading models/finalsm4hmodel/


In [ ]:
# val_df = pd.read_csv('data/orig/validation.csv')
# val_df.head()

In [7]:
tweetids = val_df['tweetid'].tolist()
texts = val_df['text'].tolist()

#for i, row in val_df.itterows():


In [ ]:
def get_top_cat(doc):
    """
    function takes a spacy doc object and returns a
    :param doc:
    :return: top category and max score
    """
    cats = doc.cats
    max_score = max(cats.values())
    max_cats = [k for k, v in cats.items() if v == max_score]
    max_cat = max_cats[0]
    return (max_cat, max_score)

In [8]:
val_df['cats'] = val_df['text'].map(lambda x: nlp(x).to_json().get('cats'))

In [9]:
val_cats_df = val_df.merge(val_df['cats'].apply(pd.Series), left_index=True, right_index=True)
val_cats_df['prediction'] = val_cats_df.cats.map(lambda x: max(x.items(), key=lambda y: y[1])[0])
val_cats_df = val_cats_df.drop(columns='cats')
val_cats_df

,text,metadata,label,tweetid,UNRELATED,ABUSE,MENTION,CONSUMPTION,prediction
0,can somebody tell me what morphine is for ?,{'tweetid': 1201409307167862784},MENTION,1201409307167862784,1.410154e-03,0.083361,0.874763,0.040466,MENTION
1,<number> mg . <number> of my x <number> i feel...,{'tweetid': 1200007750383738885},MENTION,1200007750383738885,1.095793e-04,0.669547,0.132934,0.197410,ABUSE
2,"oh hello crippling anxiety , let ’ s drive hom...",{'tweetid': 1199244035006902272},CONSUMPTION,1199244035006902272,2.389672e-03,0.051098,0.449078,0.497435,CONSUMPTION
3,pop a adderall we gone fuck all night,{'tweetid': 1199782125609902084},MENTION,1199782125609902084,1.009009e-07,0.756364,0.243469,0.000166,ABUSE
4,<hashtag> fake exercise facts </hashtag> a xan...,{'tweetid': 1199783941764517889},MENTION,1199783941764517889,3.177746e-05,0.014868,0.967707,0.017393,MENTION
...,...,...,...,...,...,...,...,...,...
2630,<user> that was not annie hall or diane keaton...,{'tweetid': 1200980670782300160},MENTION,1200980670782300160,3.268142e-04,0.010695,0.947282,0.041696,MENTION
2631,<user> suboxone for opiate dependent individua...,{'tweetid': 1199509721868374022},MENTION,1199509721868374022,2.349959e-07,0.002282,0.996483,0.001235,MENTION
2632,small brain : love lil pump med brain : xanax ...,{'tweetid': 1198691681119490050},MENTION,1198691681119490050,2.855488e-02,0.004804,0.964649,0.001993,MENTION
2633,<user> do they have a physician ? many will gi...,{'tweetid': 1200884551108714497},MENTION,1200884551108714497,3.453002e-07,0.019952,0.953284,0.026764,MENTION


In [14]:
val_cats_df[['tweetid','text', 'label', 'UNRELATED', 'ABUSE', 'MENTION',
       'CONSUMPTION', 'prediction']].to_excel('val-tweets-prediction-model.xlsx', index=False)

In [10]:
import sklearn.metrics as sklm

print(sklm.classification_report(y_pred=val_cats_df['prediction'], y_true=val_cats_df['label']))

              precision    recall  f1-score   support

       ABUSE       0.74      0.64      0.68       448
 CONSUMPTION       0.79      0.86      0.82       730
     MENTION       0.88      0.88      0.88      1353
   UNRELATED       0.93      0.91      0.92       104

    accuracy                           0.83      2635
   macro avg       0.83      0.82      0.83      2635
weighted avg       0.83      0.83      0.83      2635



In [ ]:
#sklm.average_precision_score(y_score=val_cats_df['prediction'], y_true=val_cats_df['label'])

In [16]:
val_cats_df[['tweetid','text', 'label', 'UNRELATED', 'ABUSE', 'MENTION',
       'CONSUMPTION', 'prediction']].to_csv('val-tweets-prediction-model.csv', index=False)

In [ ]:
#val_cats_df.to_csv('task4.csv', index=False)

In [ ]:
pd.set_option('max_colwidth', -1)

task4_test_df.merge(val_cats_df)[['tweetid', 'Tweet', 'text', 'ABUSE', 'UNRELATED', 'MENTION', 'CONSUMPTION', 'prediction']]

In [ ]:
task4_test_df = pd.read_csv('/Users/user/Downloads/task4_test_participant.csv')
task4_test_df

In [ ]:
dfNEW.columns

In [ ]:
val_cats_df['tweetid', 'text']

In [ ]:
docs = list(nlp.pipe(texts))


In [ ]:
docs[0].to_json().get('text')

In [ ]:
docs[0].to_json().get('cats')

In [ ]:
get_top_cat(docs[0])

In [ ]:
#tweetids

class_map = {
    'ABUSE': 'a',
    'MENTION': 'm',
    'CONSUMPTION': 'c',
    'UNRELATED': 'u'
}

In [ ]:
#docs

In [ ]:
#cats_list = [{'tweetid':tweet_id, 'Class': class_map.get(get_top_cat(doc)[0])} for (tweet_id,doc) in zip(tweetids,docs)] #[{'tweetid':tweet_id, 'Class': class_map.get(get_top_cat(doc)[0])} for (tweet_id,doc) in zip(tweetids,docs)]


In [ ]:
validation_preds_df = pd.DataFrame(cats_list)
validation_preds_df.head()

In [ ]:
! rm prediction_task4.csv.zip

In [ ]:
validation_preds_df.to_csv('prediction_task4.csv', index=False)

In [ ]:
import glob

glob.glob('/Users/user/Documents/sm*')

In [17]:
import jsonlines

with jsonlines.open('data/forprodigy/augmented-data-test_05-03-20-22-06-49.jsonl', 'r') as reader:
    val_dat = [obj for obj in reader]

In [18]:
val_df = pd.DataFrame(val_dat)
val_df

,text,metadata,label
0,<user> always preferred tramadol personally 🤪,{},CONSUMPTION
1,<user> oh my god so much is happening . <repea...,{},MENTION
2,<user> diamond dozen down day <number> of a lo...,{},CONSUMPTION
3,about to get lit and by lit i mean take <numbe...,{},ABUSE
4,forgot my pregabalin 🙄 basically means that th...,{},CONSUMPTION
...,...,...,...
2630,<number> adderall i am rollin ! <repeated>,{},ABUSE
2631,"<user> quetiapine , lamotrigin , clonazepam an...",{},MENTION
2632,<user> i bet rubio is relieved to get reacquai...,{},MENTION
2633,<user> they ’ re on <number> xanax,{},MENTION


In [ ]:
import spacy
# model loaded here
nlp = spacy.load('models/finalsm4hmodel/')